In [26]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np

# Define initial resources and expected allocations
total_food = 6000  # in metric tons
total_cash = 2000000  # in dollars

expected_allocations = {
    "Smallholder Farmers": {"food": 0.8, "cash": 0.2},
    "Pastoralists": {"food": 0.3, "cash": 0.7},
    "Urban Poor": {"food": 0.0, "cash": 1.0},
    "IDPs": {"food": 0.5, "cash": 0.5},
}

population_groups = {
    "Smallholder Farmers": 200000,
    "Pastoralists": 150000,
    "Urban Poor": 100000,
    "IDPs": 50000,
}

# Sliders for user input (percentages)
sliders_food = {
    group: widgets.IntSlider(min=0, max=100, step=1, description=group) for group in population_groups
}
sliders_cash = {
    group: widgets.IntSlider(min=0, max=100, step=1, description=group) for group in population_groups
}
submit_button = widgets.Button(description="Submit Allocation")

output = widgets.Output()
total_food_label = widgets.Label()
total_cash_label = widgets.Label()

# Function to update total percentage labels dynamically
def update_totals(*args):
    total_food_percentage = sum(slider.value for slider in sliders_food.values())
    total_cash_percentage = sum(slider.value for slider in sliders_cash.values())
    total_food_label.value = f"Total Food Allocation: {total_food_percentage}% (Max: 100%)"
    total_cash_label.value = f"Total Cash Allocation: {total_cash_percentage}% (Max: 100%)"
    
    # Disable the submit button if limits are exceeded
    if total_food_percentage > 100 or total_cash_percentage > 100:
        submit_button.disabled = True
        submit_button.description = "Adjust Allocations"
    else:
        submit_button.disabled = False
        submit_button.description = "Submit Allocation"

# Attach the update function to each slider
for slider in sliders_food.values():
    slider.observe(update_totals, names="value")
for slider in sliders_cash.values():
    slider.observe(update_totals, names="value")

# Function to handle user input and display results
def handle_submit(b):
    clear_output(wait=True)
    with output:
        # Collect user inputs as absolute allocations
        user_allocations = {
            group: {
                "food": sliders_food[group].value / 100 * total_food,
                "cash": sliders_cash[group].value / 100 * total_cash,
            }
            for group in population_groups
        }
        
        # Display user allocations
        print("Your Allocations:")
        for group, alloc in user_allocations.items():
            print(f"{group}: {alloc['food']} metric tons of food, ${alloc['cash']} cash")
        
        # Calculate expected absolute allocations based on expected percentages
        expected_abs_allocations = {
            group: {
                "food": expected_allocations[group]["food"] * total_food,
                "cash": expected_allocations[group]["cash"] * total_cash,
            }
            for group in population_groups
        }
        
        # Plotting comparison of user vs expected allocations
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        bar_width = 0.35
        index = np.arange(len(population_groups))
        
        # Food allocation comparison
        user_food = [user_allocations[group]["food"] for group in population_groups]
        expected_food = [expected_abs_allocations[group]["food"] for group in population_groups]
        
        axes[0].bar(index, user_food, bar_width, label="User Allocation")
        axes[0].bar(index + bar_width, expected_food, bar_width, label="Expected Allocation")
        axes[0].set_title("Food Allocation Comparison")
        axes[0].set_ylabel("Metric Tons")
        axes[0].set_xticks(index + bar_width / 2)
        axes[0].set_xticklabels(population_groups.keys(), rotation=45)
        axes[0].legend()
        
        # Cash allocation comparison
        user_cash = [user_allocations[group]["cash"] for group in population_groups]
        expected_cash = [expected_abs_allocations[group]["cash"] for group in population_groups]
        
        axes[1].bar(index, user_cash, bar_width, label="User Allocation")
        axes[1].bar(index + bar_width, expected_cash, bar_width, label="Expected Allocation")
        axes[1].set_title("Cash Allocation Comparison")
        axes[1].set_ylabel("Dollars")
        axes[1].set_xticks(index + bar_width / 2)
        axes[1].set_xticklabels(population_groups.keys(), rotation=45)
        axes[1].legend()
        
        plt.tight_layout()
        plt.show()

# Link button click to function
submit_button.on_click(handle_submit)

# Display interface with grouped sliders and total labels
food_box = widgets.VBox([widgets.HTML("<h4>Food Allocation (% of total)</h4>")] + list(sliders_food.values()))
cash_box = widgets.VBox([widgets.HTML("<h4>Cash Allocation (% of total)</h4>")] + list(sliders_cash.values()))
display(widgets.VBox([
    widgets.HTML("<h3>Allocate Resources for Drought Response</h3>"),
    widgets.HBox([food_box, cash_box]),
    total_food_label,
    total_cash_label,
    submit_button,
    output
]))

# Initialize total labels
update_totals()
